In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split as tts
from sklearn.preprocessing import MinMaxScaler
from sklearn import metrics
import matplotlib.pyplot as plt

In [ ]:
data_train = pd.read_csv("/kaggle/input/home-credit-default-risk/application_train.csv",index_col="SK_ID_CURR")
data_test = pd.read_csv("/kaggle/input/home-credit-default-risk/application_test.csv",index_col="SK_ID_CURR")

In [ ]:
for x in range(len(data_train.columns)):
  print(data_train.iloc[:,x].name, 
        data_train.iloc[:,x].dtypes,
        )
  pass
print(data_train.shape)

In [ ]:
list_dummy = data_train.select_dtypes("object").columns.tolist()

list_dummy

In [ ]:
train_dummies = pd.get_dummies(data_train[list_dummy])
test_dummies = pd.get_dummies(data_test[list_dummy])

In [ ]:
#difference between 2 dummies for sync 
diff = list(set(train_dummies) - set(test_dummies))
for x in diff:
  test_dummies[x] = np.nan  
  pass

train_dummies = train_dummies.reindex(sorted(train_dummies.columns), axis=1)
test_dummies = test_dummies.reindex(sorted(test_dummies.columns), axis=1)

In [ ]:
data_train_num = data_train.select_dtypes(exclude=["object"])
data_train_num = data_train_num.fillna(data_train_num.mean())
data_test_num = data_test.select_dtypes(exclude=["object"])
data_test_num = data_test_num.fillna(data_test_num.mean())

In [ ]:
data_train = pd.concat([data_train_num,
                                    train_dummies], axis = 1 )
data_test = pd.concat([data_test_num,
                                    test_dummies], axis = 1)

In [ ]:
distr = data_train.TARGET.value_counts()
distr.plot.bar(x='class', y='val', rot=0)

In [ ]:
print(data_train.TARGET.mean()*100, "percent having a bad loan")

In [ ]:
#fill empty values with 0
data_train = data_train.fillna(0)
data_test = data_test.fillna(0)

#split the data to majority and minority class 
data_train_good_loan = data_train[data_train.TARGET==0]
data_train_bad_loan = data_train[data_train.TARGET==1]




In [ ]:
print("splitted to",data_train_good_loan.shape, "<==good loan", 
      data_train_bad_loan.shape, "<==bad loan")

#downsample the majority class
data_train_good_loan_ds = data_train_good_loan.sample(len(data_train_bad_loan.TARGET))

print("now become",data_train_good_loan_ds.shape,"<==good loan",
      data_train_bad_loan.shape, "<==bad loan")

#now we need to combine the data again

data_downsample = pd.concat([data_train_good_loan_ds,data_train_bad_loan])

print("combined to ", data_downsample.shape)
print("average =", data_downsample.TARGET.mean())

print("splitted to",data_train_good_loan.shape, "<==good loan", 
      data_train_bad_loan.shape, "<==bad loan")

#upsample the majority class
data_train_bad_loan_us = data_train_bad_loan.sample(len(data_train_good_loan.TARGET),replace=True)

print("now become",data_train_good_loan.shape,"<==good loan",
      data_train_bad_loan_us.shape, "<==bad loan")

#now we need to combine the data again

data_upsample = pd.concat([data_train_good_loan,data_train_bad_loan_us])

print("combined to ", data_upsample.shape)
print("average =", data_upsample.TARGET.mean())

distr_us = data_upsample.TARGET.value_counts()
distr_us.plot.bar(x='class', y='val', rot=0)

In [ ]:
distr_ds = data_downsample.TARGET.value_counts()
distr_ds.plot.bar(x='class', y='val', rot=0)

In [ ]:
data_x = data_downsample.drop(columns=["TARGET"])
data_y = data_downsample.TARGET

print(data_x.shape,data_y.shape)

data_x = data_upsample.drop(columns=["TARGET"])
data_y = data_upsample.TARGET

print(data_x.shape,data_y.shape)

In [ ]:
#split data to train and validation

x_train, x_validate, y_train, y_validate = tts(data_x, data_y,
                                               test_size = 0.2,
                                               #0.8 train : 0.2 validation
                                               shuffle=True,
                                               random_state = 42, 
                                               #use seed for reproducability and of course
                                               #the answer of life, universe, and everything
                                               stratify=data_y
                                               #stratify y for proportional distribution
                                               )

print(x_train.shape,y_train.shape)
print(y_train.mean(), y_validate.mean())

In [ ]:
scale = MinMaxScaler()
x_train_std = scale.fit_transform(x_train)
x_validate_std = scale.fit_transform(x_validate)
x_test_std = scale.fit_transform(data_test)

print(x_train_std.shape,x_validate_std.shape,x_test_std.shape)

In [ ]:
#convert to numpy
y_train = y_train.to_numpy()
y_validate = y_validate.to_numpy()

In [ ]:
LogReg = LogisticRegression(random_state=0).fit(x_train_std, y_train)
logpred = LogReg.predict_proba(x_validate_std).T[1]

In [ ]:
false_pos_logreg, true_pos_logreg, thresholds = metrics.roc_curve(y_validate, logpred)
auc_logreg = metrics.auc(false_pos_logreg, true_pos_logreg)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
randforest = RandomForestClassifier(n_estimators=100).fit(x_train_std, y_train)
randforestpred = randforest.predict_proba(x_validate_std).T[1]

In [ ]:
false_pos_rf, true_pos_rf, thresholds = metrics.roc_curve(y_validate, randforestpred)
auc_rf = metrics.auc(false_pos_rf, true_pos_rf)

In [ ]:
import lightgbm as lgb
lgbm = lgb.LGBMClassifier(boosting_type= 'goss', n_estimators=10000, objective = 'binary', 
                          max_depth = 40, num_threads = 4, learning_rate = 0.05, 
                          bagging_fraction = 1, random_state = 42, verbose =1)
lgbm.fit(x_train_std, y_train)
lgbmpred = lgbm.predict_proba(x_validate_std).T[1]

In [ ]:
false_pos_lgbm, true_pos_lgbm, thresholds = metrics.roc_curve(y_validate, lgbmpred)
auc_lgbm = metrics.auc(false_pos_lgbm, true_pos_lgbm)

In [ ]:
plt.plot(false_pos_logreg, true_pos_logreg, linestyle='-', label='log (auc = %0.3f)' % auc_logreg)
plt.plot(false_pos_rf, true_pos_rf, linestyle='-', label='rf (auc = %0.3f)' % auc_rf)
plt.plot(false_pos_lgbm, true_pos_lgbm, linestyle='-', label='lgbm (auc = %0.3f)' % auc_lgbm)
plt.xlabel('False Positive Rate -->')
plt.ylabel('True Positive Rate -->')
plt.legend()

plt.show()

In [ ]:
res = LogReg.predict_proba(x_test_std).T[1]
index = list(data_test.index)
submit = pd.DataFrame([index,res]).T

submit.columns = ["SK_ID_CURR", "TARGET"]
submit["SK_ID_CURR"] = submit['SK_ID_CURR'].astype(int)
submit = submit.set_index("SK_ID_CURR", drop= True)

submit = submit[~submit.index.duplicated(keep='first')]
submit



In [ ]:
submit.to_csv("submit.csv")